In [1]:
#import re
# Affiche le répertoire courant
#current_directory = os.getcwd()
#print("Répertoire courant :", current_directory)
#import os
#os.chdir('/Users/sodagayethiam/Documents/Formations/Parcours_data_engineer/projet_jobmarket/JobMarket/processing')


Répertoire courant : /Users/sodagayethiam/Documents/Formations/Parcours_data_engineer/projet_jobmarket/JobMarket/processing


In [1]:
import sys
sys.path.append('../')
import glob 

import pandas as pd
from tqdm import tqdm
tqdm.pandas()  # Initialiser tqdm pour pandas
from common.utils import translate_to_english
from common.utils import clean_text
from common.utils import extract_salary_info
from common.utils import get_infos_location

import pickle
from datetime import datetime
#pd.set_option('display.max_columns', None)  # Affiche toutes les colonnes
#pd.set_option('display.max_colwidth', None)  # Affiche tout le contenu des cellules


In [2]:
import glob 
raw_path = '../outputs/raw/'

jobs_adzuna_pkl = glob.glob(f'{raw_path}*jobs_adzuna*.pkl')
jobs_adzuna = pd.concat([pd.read_pickle(file) for file in jobs_adzuna_pkl], axis=0)

jobs_themuse_pkl = glob.glob(f'{raw_path}*jobs_muse*.pkl')
jobs_themuse = pd.concat([pd.read_pickle(file) for file in jobs_themuse_pkl], axis=0)

jobs_linkedin_pkl = glob.glob(f'{raw_path}*jobs_linkedin*.pkl')
jobs_linkedin = pd.concat([pd.read_pickle(file) for file in jobs_linkedin_pkl], axis=0)

jobs_wttj_pkl  = glob.glob(f'{raw_path}*jobs_wttj*.pkl')
jobs_wttj = pd.concat([pd.read_pickle(file) for file in jobs_wttj_pkl], axis=0)

print('taille jobs_adzuna:',jobs_adzuna.shape)
print('taille jobs_linkedin:',jobs_linkedin.shape)
print('taille jobs_themuse:',jobs_themuse.shape)
print('taille jobs_wttj:',jobs_wttj.shape)


taille jobs_adzuna: (24600, 23)
taille jobs_linkedin: (7270, 15)
taille jobs_themuse: (1980, 16)
taille jobs_wttj: (13994, 21)


In [13]:

state
city
latitude
longitude

(24600, 23)
(24600, 23)
(24600, 23)


In [3]:
for column in jobs_adzuna.columns:
    print(column, type(jobs_adzuna[column].iloc[0]))
jobs_adzuna.head()



contract_type <class 'str'>
id <class 'str'>
longitude <class 'numpy.float64'>
latitude <class 'numpy.float64'>
redirect_url <class 'str'>
salary_min <class 'numpy.float64'>
__CLASS__ <class 'str'>
description <class 'str'>
adref <class 'str'>
salary_max <class 'numpy.float64'>
title <class 'str'>
created <class 'str'>
salary_is_predicted <class 'str'>
location.area <class 'list'>
location.__CLASS__ <class 'str'>
location.display_name <class 'str'>
category.label <class 'str'>
category.__CLASS__ <class 'str'>
category.tag <class 'str'>
company.__CLASS__ <class 'str'>
company.display_name <class 'str'>
contract_time <class 'float'>
source <class 'str'>


,contract_type,id,longitude,latitude,redirect_url,salary_min,__CLASS__,description,adref,salary_max,...,location.area,location.__CLASS__,location.display_name,category.label,category.__CLASS__,category.tag,company.__CLASS__,company.display_name,contract_time,source
0,permanent,4861152531,-111.864161,40.759900,https://www.adzuna.com/details/4861152531?utm_...,0.0,Adzuna::API::Response::Job,Are you a dedicated Internal Medicine Physicia...,eyJhbGciOiJIUzI1NiJ9.eyJzIjoiT3RxS2l2MTQ3eEc2e...,225000.0,...,"[US, Utah, Salt Lake County, Fort Douglas]",Adzuna::API::Response::Location,"Fort Douglas, Salt Lake County",Healthcare & Nursing Jobs,Adzuna::API::Response::Category,healthcare-nursing-jobs,Adzuna::API::Response::Company,AnyPlace MD,NaN,adzuna
1,permanent,4736389699,-122.639261,45.446231,https://www.adzuna.com/details/4736389699?utm_...,26400.0,Adzuna::API::Response::Job,"As a Customer Service Representative, you'll b...",eyJhbGciOiJIUzI1NiJ9.eyJzIjoiT3RxS2l2MTQ3eEc2e...,45600.0,...,"[US, Oregon, Clackamas County, Milwaukie]",Adzuna::API::Response::Location,"Milwaukie, Clackamas County",Customer Services Jobs,Adzuna::API::Response::Category,customer-services-jobs,Adzuna::API::Response::Company,Ada Business Solutions,full_time,adzuna
2,permanent,4851460203,-87.813410,42.693660,https://www.adzuna.com/details/4851460203?utm_...,225000.0,Adzuna::API::Response::Job,Are you a Physician ready for a fresh start in...,eyJhbGciOiJIUzI1NiJ9.eyJzIjoiT3RxS2l2MTQ3eEc2e...,300000.0,...,"[US, Wisconsin, Racine County, Racine]",Adzuna::API::Response::Location,"Racine, Racine County",Healthcare & Nursing Jobs,Adzuna::API::Response::Category,healthcare-nursing-jobs,Adzuna::API::Response::Company,AnyPlace MD,NaN,adzuna
3,permanent,4849468031,-122.612314,45.365676,https://www.adzuna.com/details/4849468031?utm_...,26400.0,Adzuna::API::Response::Job,"As a Customer Service Representative, you'll b...",eyJhbGciOiJIUzI1NiJ9.eyJpIjoiNDg0OTQ2ODAzMSIsI...,45600.0,...,"[US, Oregon, Clackamas County, West Linn]",Adzuna::API::Response::Location,"West Linn, Clackamas County",Customer Services Jobs,Adzuna::API::Response::Category,customer-services-jobs,Adzuna::API::Response::Company,Ada Business Solutions,full_time,adzuna
4,permanent,4851460201,-122.576100,45.440340,https://www.adzuna.com/details/4851460201?utm_...,225000.0,Adzuna::API::Response::Job,We are seeking a dedicated Physician to serve ...,eyJhbGciOiJIUzI1NiJ9.eyJpIjoiNDg1MTQ2MDIwMSIsI...,300000.0,...,"[US, Oregon, Clackamas County, Clackamas]",Adzuna::API::Response::Location,"Clackamas, Clackamas County",Healthcare & Nursing Jobs,Adzuna::API::Response::Category,healthcare-nursing-jobs,Adzuna::API::Response::Company,AnyPlace MD,NaN,adzuna


0    Not found
1    Not found
2    Not found
3    Not found
4    Not found
5    Not found
6    Not found
dtype: object

In [4]:
locat='Fort Douglas, Salt Lake County'
get_infos_location(locat)

0          United States
1                   Utah
2         Salt Lake City
3                  84112
4             40.7653418
5    -111.83256760529602
6              US Dollar
dtype: object

In [3]:
jobs_adzuna_selected = jobs_adzuna[['title','description','contract_type', 'contract_time',
'company.display_name', 'created','location.display_name','category.label', 'salary_max' ,'salary_min', #'location.area',
'redirect_url','source']].rename(columns={'title': 'job_title', 'description': 'job_description',
'company.display_name': 'company_name','created': 'publication_datetime','location.display_name': 'location','category.label': 'category',
'redirect_url': 'offer_link'})
#'location.area': 'locations_company',

jobs_adzuna_selected['publication_date'] = pd.to_datetime(jobs_adzuna_selected['publication_datetime']).dt.strftime("%Y-%m-%d")

jobs_adzuna_selected['publication_time'] = jobs_adzuna_selected['publication_datetime'].apply(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%SZ").time())
#jobs_adzuna_selected['country']='US'
jobs_adzuna_selected['industry']='not available'
#jobs_adzuna_selected['currency']='dollars'
jobs_adzuna_selected['levels']='not available'


#jobs_adzuna_selected['locations_company'] = jobs_adzuna_selected['locations_company'].astype(str)
#jobs_adzuna_selected['locations_company'] = jobs_adzuna_selected['locations_company'].str.replace(r'[\[\]]', '', regex=True)
jobs_adzuna_selected.head()

print( 'taille jobadzuna avant dedoubloonnage', jobs_adzuna_selected.shape)
#companies['contents'] = companies['description'].apply(clean_text)
jobs_adzuna_selected=jobs_adzuna_selected.drop_duplicates()
jobs_adzuna_selected[['country', 'state', 'city', 'postcode', 'latitude', 'longitude', 'currency_name']] = jobs_adzuna_selected['location'].apply(get_infos_location)

print( 'taille jobadzuna apres dedoubloonnage', jobs_adzuna_selected.shape)


taille jobadzuna avant dedoubloonnage (24600, 16)


KeyboardInterrupt: 

In [7]:
jobs_adzuna_selected.head()

,job_title,job_description,contract_type,contract_time,company_name,publication_datetime,location,category,salary_max,salary_min,offer_link,source,publication_date,publication_time,country,industry,currency,levels
0,Corrections LPN - Night Shift (Licensed Practi...,We're helping our client to bring on dedicated...,contract,NaN,AnyPlace MD,2024-10-21T19:21:54Z,"Phoenix, Maricopa County",Healthcare & Nursing Jobs,74880.00,67200.00,https://www.adzuna.com/details/4909432595?utm_...,adzuna,2024-10-21,19:21:54,US,not available,dollars,not available
1,Clinical Lab Technologist - Blood Bank,We are seeking a Clinical Lab Technologist for...,permanent,full_time,AnyPlace MD,2024-08-30T05:14:45Z,"Van Cott, Bronx",Scientific & QA Jobs,109000.00,88000.00,https://www.adzuna.com/details/4844860393?utm_...,adzuna,2024-08-30,05:14:45,US,not available,dollars,not available
2,Customer Service/Sales Rep,Would being part of a cutting edge technology ...,permanent,full_time,Ada Business Solutions,2024-06-14T09:33:24Z,"El Paso, El Paso County",Sales Jobs,57600.00,28800.00,https://www.adzuna.com/details/4736389568?utm_...,adzuna,2024-06-14,09:33:24,US,not available,dollars,not available
3,Middle School Substitute Teacher,Step in for absent teachers with Swing Educati...,NaN,NaN,Swing Education,2024-09-06T20:19:29Z,"Hackensack, Bergen County",Teaching Jobs,42157.43,42157.43,https://www.adzuna.com/details/4855075568?utm_...,adzuna,2024-09-06,20:19:29,US,not available,dollars,not available
4,"Substitute Teacher, create your own scedule",Step in for absent teachers with Swing Educati...,NaN,NaN,Swing Education,2024-11-01T19:44:20Z,"Melrose Park, Cook County",Teaching Jobs,41046.05,41046.05,https://www.adzuna.com/details/4922651186?utm_...,adzuna,2024-11-01,19:44:20,US,not available,dollars,not available


In [8]:
##jobs_adzuna_selected[jobs_adzuna_selected['job_title']=='Clinical Lab Technologist - Blood Bank']
#duplicates = jobs_adzuna_selected[jobs_adzuna_selected.duplicated()]
#duplicates[duplicates['job_title']=='Clinical Lab Technologist - Blood Bank']

In [9]:
jobs_themuse.head()

,contents,name,type,publication_date,short_name,model_type,id,locations,categories,tags,refs.landing_page,company.id,company.short_name,company.name,levels.name,levels.short_name
0,<p><strong>Gemeinsam machen wir den Unterschie...,Freileitungsmonteur (w/m/d) Fahrleitungs- und ...,external,2023-01-24T00:42:10Z,freileitungsmonteur-wmd-fahrleitungs-und-strom...,jobs,10536487,"[Bad Mergentheim, Germany]",Computer and IT,[],https://www.themuse.com/jobs/siemens/freileitu...,11913,siemens,Siemens,Mid Level,mid
1,<p><strong>Mechanical Assembler</strong><br><b...,Mechanical Assembler - 1st Shift,external,2023-04-26T23:41:35Z,mechanical-assembler-1st-shift-0a7d96,jobs,11259124,"[Clayton, NC]",Software Engineering,[],https://www.themuse.com/jobs/siemens/mechanica...,11913,siemens,Siemens,Senior Level,senior
2,<p>Bring your heart to CVS Health. Every one o...,Pharmacy Technician,external,2024-06-06T12:25:46Z,pharmacy-technician-4dfa8b,jobs,14569255,"[Rockville, MD]",Healthcare,[],https://www.themuse.com/jobs/cvshealth/pharmac...,12096,cvshealth,CVS Health,Senior Level,senior
3,<p><strong>Who designs your future? You do.</s...,Sr. Systems Specialist - Smart Buildings / Aut...,external,2023-05-03T23:45:16Z,sr-systems-specialist-smart-buildings-automati...,jobs,11034620,"[Waterloo, IA]",Design and UX,[],https://www.themuse.com/jobs/siemens/sr-system...,11913,siemens,Siemens,Senior Level,senior
4,<p>Bring your heart to CVS Health. Every one o...,Pharmacy Technician Front End - Full Time,external,2024-06-06T12:33:58Z,pharmacy-technician-front-end-full-time-416939,jobs,14444118,"[Edison, NJ]",Healthcare,[],https://www.themuse.com/jobs/cvshealth/pharmac...,12096,cvshealth,CVS Health,Mid Level,mid


In [10]:
jobs_theMuse_selected = jobs_themuse[['name','contents','type', 'company.name',
'publication_date', 'locations','categories','refs.landing_page','levels.name']].rename(columns={'name': 'job_title', 'contents': 'job_description',
'type': 'contract_type','company.name': 'company_name','publication_date': 'publication_datetime','locations': 'location',
'categories': 'category','refs.landing_page': 'offer_link','levels.name':'levels'})

jobs_theMuse_selected['publication_date'] = pd.to_datetime(jobs_theMuse_selected['publication_datetime']).dt.strftime("%Y-%m-%d")
jobs_theMuse_selected['publication_time'] = jobs_theMuse_selected['publication_datetime'].apply(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%SZ").time())

jobs_theMuse_selected['country']='a travailler'
jobs_theMuse_selected['industry']='not available'
jobs_theMuse_selected['currency']='not available'
jobs_theMuse_selected['contract_time']='not available'
jobs_theMuse_selected['salary_max']=0
jobs_theMuse_selected['salary_min']=0

jobs_theMuse_selected['location'] = jobs_theMuse_selected['location'].astype(str)
jobs_theMuse_selected['location'] = jobs_theMuse_selected['location'].str.replace(r'[\[\]]', '', regex=True)
#jobs_theMuse_selected['locations_company']=jobs_theMuse_selected['location']

jobs_theMuse_selected['job_description'] = jobs_theMuse_selected['job_description'].apply(clean_text)

jobs_theMuse_selected["source"] = "TheMuse"

jobs_theMuse_selected.head()


,job_title,job_description,contract_type,company_name,publication_datetime,location,category,offer_link,levels,publication_date,publication_time,country,industry,currency,contract_time,salary_max,salary_min,source
0,Freileitungsmonteur (w/m/d) Fahrleitungs- und ...,<p><strong>Gemeinsam machen wir den Unterschie...,external,Siemens,2023-01-24T00:42:10Z,"'Bad Mergentheim, Germany'",Computer and IT,https://www.themuse.com/jobs/siemens/freileitu...,Mid Level,2023-01-24,00:42:10,a travailler,not available,not available,not available,0,0,TheMuse
1,Mechanical Assembler - 1st Shift,<p><strong>Mechanical Assembler</strong><br><b...,external,Siemens,2023-04-26T23:41:35Z,"'Clayton, NC'",Software Engineering,https://www.themuse.com/jobs/siemens/mechanica...,Senior Level,2023-04-26,23:41:35,a travailler,not available,not available,not available,0,0,TheMuse
2,Pharmacy Technician,<p>Bring your heart to CVS Health. Every one o...,external,CVS Health,2024-06-06T12:25:46Z,"'Rockville, MD'",Healthcare,https://www.themuse.com/jobs/cvshealth/pharmac...,Senior Level,2024-06-06,12:25:46,a travailler,not available,not available,not available,0,0,TheMuse
3,Sr. Systems Specialist - Smart Buildings / Aut...,<p><strong>Who designs your future? You do.</s...,external,Siemens,2023-05-03T23:45:16Z,"'Waterloo, IA'",Design and UX,https://www.themuse.com/jobs/siemens/sr-system...,Senior Level,2023-05-03,23:45:16,a travailler,not available,not available,not available,0,0,TheMuse
4,Pharmacy Technician Front End - Full Time,<p>Bring your heart to CVS Health. Every one o...,external,CVS Health,2024-06-06T12:33:58Z,"'Edison, NJ'",Healthcare,https://www.themuse.com/jobs/cvshealth/pharmac...,Mid Level,2024-06-06,12:33:58,a travailler,not available,not available,not available,0,0,TheMuse


In [11]:

#df[['currency', 'min_salary', 'max_salary']] = df['salary'].apply(extract_salary_info)
jobs_wttj.head()

,title,company,type_company,site_company,type_contract,salary,contents,date_public,monthsOfExperience,employmentType,...,qualifications,validThrough,logo,addressCountry,postalCode,addressLocality,streetAddress,industry,addressRegion,link_job
0,Manager Conseil en transformation data (H/F),iQo,Organization,https://www.iqo.eu/,Permanent contract,Salary:Not specified,<h3>🚀 <strong>iQo – le conseil accélérateur de...,2024-08-22T13:12:43Z,Not specified,FULL_TIME,...,,2024-11-20T13:12:43.000Z,https://cdn-images.welcometothejungle.com/C0fD...,FR,75008,Paris,"91 Rue du Faubourg Saint-Honoré, 75008 Paris, ...","Organization / Management, Strategy, Change Ma...",Paris,https://www.welcometothejungle.com/en/companie...
1,Consultant.e senior Stratégie & Conseil,iQo,Organization,https://www.iqo.eu/,Permanent contract,Salary:Not specified,<h3><strong>🚀 iQo – le conseil accélérateur de...,2024-08-22T13:12:43Z,Not specified,FULL_TIME,...,,2024-11-20T13:12:43.000Z,https://cdn-images.welcometothejungle.com/C0fD...,FR,75008,Paris,"91 Rue du Faubourg Saint-Honoré, 75008 Paris, ...","Organization / Management, Strategy, Change Ma...",Paris,https://www.welcometothejungle.com/en/companie...
2,Consultant.e Senior Supply chain,iQo,Organization,https://www.iqo.eu/,Permanent contract,Salary:Not specified,<h3><strong>🚀 iQo – le conseil accélérateur de...,2024-08-22T13:12:43Z,Not specified,FULL_TIME,...,,2024-11-20T13:12:43.000Z,https://cdn-images.welcometothejungle.com/C0fD...,FR,75008,Paris,"91 Rue du Faubourg Saint-Honoré, 75008 Paris, ...","Organization / Management, Strategy, Change Ma...",Paris,https://www.welcometothejungle.com/en/companie...
3,Consultant.e Contract Management,iQo,Organization,https://www.iqo.eu/,Permanent contract,Salary:Not specified,<h3><strong>🚀 iQo – le conseil accélérateur de...,2024-08-22T13:12:43Z,Not specified,FULL_TIME,...,,2024-11-20T13:12:43.000Z,https://cdn-images.welcometothejungle.com/C0fD...,FR,75008,Paris,"91 Rue du Faubourg Saint-Honoré, 75008 Paris, ...","Organization / Management, Strategy, Change Ma...",Paris,https://www.welcometothejungle.com/en/companie...
4,Manager Conseil Services Financiers - Offre Ba...,iQo,Organization,https://www.iqo.eu/,Permanent contract,Salary:Not specified,<h3><strong>🚀 iQo – le conseil accélérateur de...,2024-08-22T13:12:43Z,Not specified,FULL_TIME,...,,2024-11-20T13:12:43.000Z,https://cdn-images.welcometothejungle.com/C0fD...,FR,75008,Paris,"91 Rue du Faubourg Saint-Honoré, 75008 Paris, ...","Organization / Management, Strategy, Change Ma...",Paris,https://www.welcometothejungle.com/en/companie...


In [12]:
jobs_wttj_selected = jobs_wttj[['title', 'company', 'type_contract', 'contents', 'date_public',
       'employmentType','addressCountry', 'addressLocality',  'industry', 'link_job', 'salary']].rename(
columns={'title': 'job_title', 'contents': 'job_description',
'employmentType': 'contract_type','company': 'company_name','date_public': 'publication_datetime','addressLocality': 'location',
' addressCountry': 'country','link_job': 'offer_link'})



jobs_wttj_selected['publication_date'] = pd.to_datetime(jobs_wttj_selected['publication_datetime']).dt.strftime("%Y-%m-%d")
jobs_wttj_selected['publication_time'] = jobs_wttj_selected['publication_datetime'].apply(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%SZ").time())
jobs_wttj_selected['industry']='not available'
jobs_wttj_selected['contract_time']='not available'
jobs_wttj_selected['category']='not available'

jobs_wttj_selected[['currency', 'salary_min', 'salary_max']] = jobs_wttj_selected['salary'].apply(extract_salary_info)
jobs_wttj_selected = jobs_wttj_selected.drop('salary', axis=1)

jobs_wttj_selected['location'] = jobs_wttj_selected['location'].astype(str)
jobs_wttj_selected['location'] = jobs_wttj_selected['location'].str.replace(r'[\[\]]', '', regex=True)
#jobs_wttj_selected['locations_company']=jobs_wttj_selected['location']

jobs_wttj_selected['job_description'] = jobs_wttj_selected['job_description'].apply(clean_text)
jobs_wttj_selected["levels"] = "not available"

jobs_wttj_selected["source"] = "Wttj"
jobs_wttj_selected=jobs_wttj_selected.drop_duplicates()


jobs_wttj_selected.head()

,job_title,company_name,type_contract,job_description,publication_datetime,contract_type,addressCountry,location,industry,offer_link,publication_date,publication_time,contract_time,category,currency,salary_min,salary_max,levels,source
0,Manager Conseil en transformation data (H/F),iQo,Permanent contract,<h3>🚀 <strong>iQo – le conseil accélérateur de...,2024-08-22T13:12:43Z,FULL_TIME,FR,Paris,not available,https://www.welcometothejungle.com/en/companie...,2024-08-22,13:12:43,not available,not available,Not,0.0,0.0,not available,Wttj
1,Consultant.e senior Stratégie & Conseil,iQo,Permanent contract,<h3><strong>🚀 iQo – le conseil accélérateur de...,2024-08-22T13:12:43Z,FULL_TIME,FR,Paris,not available,https://www.welcometothejungle.com/en/companie...,2024-08-22,13:12:43,not available,not available,Not,0.0,0.0,not available,Wttj
2,Consultant.e Senior Supply chain,iQo,Permanent contract,<h3><strong>🚀 iQo – le conseil accélérateur de...,2024-08-22T13:12:43Z,FULL_TIME,FR,Paris,not available,https://www.welcometothejungle.com/en/companie...,2024-08-22,13:12:43,not available,not available,Not,0.0,0.0,not available,Wttj
3,Consultant.e Contract Management,iQo,Permanent contract,<h3><strong>🚀 iQo – le conseil accélérateur de...,2024-08-22T13:12:43Z,FULL_TIME,FR,Paris,not available,https://www.welcometothejungle.com/en/companie...,2024-08-22,13:12:43,not available,not available,Not,0.0,0.0,not available,Wttj
4,Manager Conseil Services Financiers - Offre Ba...,iQo,Permanent contract,<h3><strong>🚀 iQo – le conseil accélérateur de...,2024-08-22T13:12:43Z,FULL_TIME,FR,Paris,not available,https://www.welcometothejungle.com/en/companie...,2024-08-22,13:12:43,not available,not available,Not,0.0,0.0,not available,Wttj


In [13]:
#jobs_linkedin.link.unique()
jobs_linkedin.head()

,title,company,location,link,salary,salary_min,salary_max,seniority_level,employment_type,job_function,job_industry,posted_time,company_link,source,ingest_date
0,Healthcare Scheduler - Fully Remote,Vigilance Health,United States,https://www.linkedin.com/jobs/view/healthcare-...,$18.00 - $19.00,$18.00,$19.00,Entry level,Full-time,Health Care Provider,Hospitals and Health Care,2024-11-02,https://www.linkedin.com/company/vigilance-hea...,linkedin,2024-11-02 19:54:25.139908
1,Marketing Assistant,PyjamaHR,"New York, NY",https://www.linkedin.com/jobs/view/marketing-a...,,,,Entry level,Full-time,Administrative,Professional Training and Coaching,2024-11-02,https://www.linkedin.com/company/computercore?...,linkedin,2024-11-02 19:54:25.139908
2,Online Proofreader,IEM,United States,https://www.linkedin.com/jobs/view/online-proo...,,,,Not Applicable,Full-time,Sales and Business Development,"Retail Apparel and Fashion, Design Services, a...",2024-11-02,https://www.linkedin.com/company/beyond-yoga?t...,linkedin,2024-11-02 19:54:25.139908
3,Bagger,Jungle Jim's International Market,"Fairfield, OH",https://www.linkedin.com/jobs/view/bagger-at-j...,,,,Entry level,Part-time,Information Technology,Human Resources Services,2024-11-02,https://www.linkedin.com/company/timelyfind?tr...,linkedin,2024-11-02 19:54:25.139908
4,Data Entry Operator,Computer CORE (Community OutReach and Education),United States,https://www.linkedin.com/jobs/view/data-entry-...,$45.00 - $50.00,$45.00,$50.00,Entry level,Full-time,Other,Retail,2024-11-02,https://www.linkedin.com/company/jungle-jim's-...,linkedin,2024-11-02 19:54:25.139908


In [14]:
jobs_linkedin_selected = jobs_linkedin[['title','employment_type', 'company','salary_min','salary_max',
'posted_time','location','job_industry','link','source','seniority_level','job_function']].rename(columns={'title': 'job_title', 'employment_type': 'contract_time',
'company': 'company_name','posted_time': 'publication_datetime','job_industry': 'industry','link': 'offer_link','seniority_level':'levels','job_function':'category'})

#jobs_linkedin_selected['publication_date'] = pd.to_datetime(jobs_adzuna_selected['publication_datetime']).dt.strftime("%Y-%m-%d")
jobs_linkedin_selected['contents']='not available'
jobs_linkedin_selected['currency']='dollars'
jobs_linkedin_selected['publication_time'] =None

jobs_linkedin_selected['publication_date'] = pd.to_datetime(jobs_linkedin_selected['publication_datetime']).dt.strftime("%Y-%m-%d")

jobs_linkedin_selected['contract_type']='not available'

#jobs_linkedin_selected['locations_company']='not available'

#jobs_linkedin_selected['job_description'] = jobs_linkedin_selected['job_description'].apply(clean_text)
jobs_linkedin_selected.head()

,job_title,contract_time,company_name,salary_min,salary_max,publication_datetime,location,industry,offer_link,source,levels,category,contents,currency,publication_time,publication_date,contract_type
0,Healthcare Scheduler - Fully Remote,Full-time,Vigilance Health,$18.00,$19.00,2024-11-02,United States,Hospitals and Health Care,https://www.linkedin.com/jobs/view/healthcare-...,linkedin,Entry level,Health Care Provider,not available,a travailler,None,2024-11-02,not available
1,Marketing Assistant,Full-time,PyjamaHR,,,2024-11-02,"New York, NY",Professional Training and Coaching,https://www.linkedin.com/jobs/view/marketing-a...,linkedin,Entry level,Administrative,not available,a travailler,None,2024-11-02,not available
2,Online Proofreader,Full-time,IEM,,,2024-11-02,United States,"Retail Apparel and Fashion, Design Services, a...",https://www.linkedin.com/jobs/view/online-proo...,linkedin,Not Applicable,Sales and Business Development,not available,a travailler,None,2024-11-02,not available
3,Bagger,Part-time,Jungle Jim's International Market,,,2024-11-02,"Fairfield, OH",Human Resources Services,https://www.linkedin.com/jobs/view/bagger-at-j...,linkedin,Entry level,Information Technology,not available,a travailler,None,2024-11-02,not available
4,Data Entry Operator,Full-time,Computer CORE (Community OutReach and Education),$45.00,$50.00,2024-11-02,United States,Retail,https://www.linkedin.com/jobs/view/data-entry-...,linkedin,Entry level,Other,not available,a travailler,None,2024-11-02,not available


In [15]:
jobs1 = pd.concat([jobs_linkedin_selected, jobs_theMuse_selected], axis=0)
jobs2 = pd.concat([jobs1, jobs_adzuna_selected], axis=0)
jobs_final = pd.concat([jobs2, jobs_wttj_selected], axis=0)
jobs_final=jobs_final.drop_duplicates()
#jobs = pd.concat([jobs2, jobs_wttj_selected], axis=0)

shape=jobs_final.shape
print(f"Taille du dataframe fusionné : {shape}")
#Taille du dataframe fusionné : (16269, 21)


Taille du dataframe fusionné : (25571, 21)


In [16]:
jobs_final.head()
jobs_final['contents'] = jobs_final['contents'].progress_apply(translate_to_english)


100%|██████████| 25571/25571 [00:09<00:00, 2825.46it/s]


In [17]:
jobs_final.to_pickle('../outputs/final/jobs.pkl')


In [20]:
companies_linkedin.head()

,company_name,company_description,company_website,company_industries,company_size,company_associated_members,company_location,company_creation_date,company_specialities,source,ingest_date
0,U.S. House of Representatives,The U.S. House of Representatives is one of tw...,http://www.house.gov/,Legislative Offices,"10,001+ employees",LinkedIn members who’ve listed U.S. House of R...,"Washington, DC",None,None,linkedin,2024-11-07 18:17:14.373196
1,Relate Search,"Welcome to Relate Search, the purpose-driven s...",https://www.relatesearch.com/,Staffing and Recruiting,11-50 employees,LinkedIn members who’ve listed Relate Search a...,"Cincinnati , OH",2023,"Recruiting, Executive Search, Human Resources,...",linkedin,2024-11-07 18:17:14.373196
2,Avispa,"At Avispa, we ensure our contractors are as sa...",http://www.avispatechnology.com,Staffing and Recruiting,51-200 employees,LinkedIn members who’ve listed Avispa as their...,"Truckee, California",2023,"Recruiting, Executive Search, Human Resources,...",linkedin,2024-11-07 18:17:14.373196
3,clearer.io,"At Clearer.io, we’re on a mission to simplify ...",https://www.clearer.io/,Software Development,201-500 employees,LinkedIn members who’ve listed clearer.io as t...,"Boston, Massachusetts",2011,"Workforce Management Services, Flexible Worker...",linkedin,2024-11-07 18:17:14.373196
4,Cigna Healthcare,We are a health benefits provider that advocat...,Visit website,Hospitals and Health Care,"10,001+ employees",LinkedIn members who’ve listed Cigna Healthcar...,"Bloomfield, CT",2011,"Workforce Management Services, Flexible Worker...",linkedin,2024-11-07 18:17:14.373196


Index(['company_name', 'company_description', 'company_website',
       'company_industries', 'company_size', 'company_associated_members',
       'company_location', 'company_creation_date', 'company_specialities',
       'source', 'ingest_date'],
      dtype='object')

In [ ]:

#jupyter nbconvert --to script process_jobs.ipynb

In [9]:
import pandas as pd
import glob

linkedin_path = '../outputs/raw/'

# Lister tous les fichiers contenant 'companies_linkedin' dans leur nom
files_linkedin = glob.glob(f'{linkedin_path}*companies_linkedin*.pkl')
companies_linkedin = pd.concat([pd.read_pickle(file) for file in files_linkedin], axis=0)
companies_linkedin.shape

(251, 12)